This project is to build a web scraper that can:
* scrape all search results from real estate websites and build a databasee
* perform exploratory data analysis, and find valued propertities

The website I will be scraping is zillow(https://www.zillow.com/).

**Agenda**:
* preparing for the packages
* requests html pages of the data
* use beautiful soup to parse information
* extract specific house information
* functionally scrape house information 
* 

**Preparing for the packages**

In [328]:
from bs4 import BeautifulSoup
from requests import get
import re
import pandas as pd
import itertools
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

**Requests html pages of the data**

In [424]:
url = "https://www.zillow.com/homes/for_sale/27560_rb/2_p"
headers = ({'User-Agent':
            'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3607.0 Safari/537.36'})
response = get(url, headers = headers)

**Use beautiful soup to parse information**

In [425]:
html_soup = BeautifulSoup(response.text, 'html.parser')

In [426]:
facts = html_soup.find('div', class_='zsg-separator')
house_containers = html_soup.find_all('div', class_='zsg-photo-card-content zsg-aspect-ratio-content')

In [421]:
pattern = 'for Sale:[0-9]+'
line = re.search(pattern, facts.text).group()
ind = line.index(':')
total = int(line[ind+1:])
total

76

In [430]:
first = house_containers[4]
first.find_all('span')

[<span class="hide" itemprop="address" itemscope="" itemtype="http://schema.org/PostalAddress"><span itemprop="streetAddress">104 Carter Grove Ct</span><span itemprop="addressLocality"> MORRISVILLE </span><span itemprop="addressRegion">NC </span><span class="hide" itemprop="postalCode">27560</span></span>,
 <span itemprop="streetAddress">104 Carter Grove Ct</span>,
 <span itemprop="addressLocality"> MORRISVILLE </span>,
 <span itemprop="addressRegion">NC </span>,
 <span class="hide" itemprop="postalCode">27560</span>,
 <span itemprop="geo" itemscope="" itemtype="http://schema.org/GeoCoordinates"><meta content="35.8365" itemprop="latitude"/><meta content="-78.8686" itemprop="longitude"/></span>,
 <span class="zsg-photo-card-status"><span class="zsg-icon-for-sale"></span>House for sale</span>,
 <span class="zsg-icon-for-sale"></span>,
 <span class="zsg-photo-card-price">$475,000</span>,
 <span class="zsg-photo-card-info">4 bds <span class="interpunct">·</span> 3 ba <span class="interpunc

**Extract specific house information**

extract house price

In [428]:
var_1 = first.find_all('span')[8].text
var_1

'$314,200+'

extrace house location

In [413]:
location = first.find_all('p')[1].text
location

'107 Gratiot Dr, Morrisville, NC'

extract house size

In [414]:
size = first.find_all('span')[9].text
size.split(' · ')

['4 bds', '3 ba', '2,060 sqft']

get all the links

In [107]:
links = []
l = len(house_containers)
for i in range(l):
    for url in house_containers[i].find_all('a'):
        link = url.get('href')
        if link.startswith('/myzillow') is False:
            link = 'https://www.zillow.com' + link
            links.append(link)        

**Functionally scrape information**